# Análisis e Implementación en SparkQL

* Entrega hasta el 24 de mayo en https://forms.gle/aRiWtjayausRFCTd8

* Los dos conjuntos de datos entregados, CSV separados por coma, Evaluación del Pitch (2021.04.13 ISoftware).csv y Asistencia del Pitch (2021.04.13 ISoftware).csv proceden de dos encuestas realizadas con Google Forms.

* Los archivos proceden de las evaluaciones de los pitch (exposiciones cortas de negocio) que realizan los estudiantes de la asignatura de Ingeniería de Software. Los estudiantes están organizados en equipos. Cada equipo realiza una presentación de máximo 15 minutos, una vez inicia la presentación todos los estudiantes deben registrar su asistencia lo cual queda registrado en 'Asistencia del Pitch (2021.04.13 ISoftware).csv' incluidos los miembros del equipo.

* El archivo `Asistencia del Pitch (2021.04.13 ISoftware).csv` contiene las columnas: `"Marca temporal"` es tiempo dado en fecha y hora, `"Nombre de usuario"` es el correo electrónico del estudiante y es texto,`"Equipo al que perteneces:"` equipo de trabajo al que pertenece el estudiante también es texto y `"Equipo que va a exponer:"` equipo que el estudiante va a tender a su presentación.

* El archivo `Evaluación del Pitch (2021.04.13 ISoftware).csv` contiene las columnas `"Marca temporal"` es tiempo dado en fecha y hora, `"Nombre de usuario"` es el correo electrónico del estudiante y es texto, `"Equipo que vas a evaluar:"` equipo que ha expuesto y que va a ser evaluado por cada estudiante que no sea integrante; a continuación, se tienen las siguientes columnas que corresponden a la evaluación de los respectivos ítems:

    1. "Introducción: El equipo responde adecuadamente ¿Quiénes son y por qué están aquí?",
    2. "Equipo: El equipo responde adecuadamente ¿Quiénes están detrás de la idea y cuál es su función?",
    3. "Problema: El equipo responde adecuadamente ¿Qué problema resolverá?, ¿es realmente un problema?",
    4. "Ventajas: El equipo responde adecuadamente ¿Por qué su solución es especial?, ¿qué la hace distinta de otras?",
    5. "Solución: El equipo responde adecuadamente ¿Cómo piensa resolver el problema?",
    6. "Producto: El equipo responde adecuadamente ¿Cómo funciona el producto o servicio? Muestra algunos ejemplos.",
    7. "Tracción: El equipo responde adecuadamente si cuenta con clientes que demuestran potencial.",
    8. "Mercado: El equipo responde conoce, o por lo menos intentar predecir, el tamaño del mercado que impactará.",
    9. "Competencia: El equipo responde adecuadamente ¿Cuáles son las soluciones alternativas al problema que plantea?",
    10. "Modelo de negocio: El equipo responde adecuadamente ¿Cómo hará dinero? ",
    11. "Inversión: El equipo responde adecuadamente ¿Cuál es su presupuesto y cuánto espera ganar?",
    12. "Contacto: El equipo deja los datos al cliente y muestra cómo pueden contactarle.",
    13. "Exposición: ¿Qué tan coordinados estaban los expositores?",
    14. "Exposición: ¿Los expositores se expresaron con claridad y se hicieron entender?",
    15. "Exposición: Las diapositivas son claras y coherentes y apoyaron adecuadamente la exposición.",
    * "Suponiendo que eres inversionista, ¿Estarías dispuesto a invertir dinero en este equipo? (esta pregunta no se pondera en la nota)",
    * "Observaciones para el equipo, estas observaciones las debe considerar el equipo para mejorar la siguiente presentación."


* Cada ítem se evalúa con la siguiente escala: 0. Ausente; 1. Deficiente; 2. Regular; 3. Aceptable; 4. Bueno; 5. Excelente

* Carge los datos, cada archivo en una tabla SparkSQL y responda cada una de las consultas dadas en cada celda. Tenga en cuenta que algunas consultas pueden tener como resultado el vacío.

# Integrantes del equipo
1. Santiago Rincón Perez
2. Julian Saldarriaga Blandon
3. Angie Velasquez Velez
4. Harvey Rodríguez Gil
5.

In [1]:
# Agrege acá el código para importar las librerias

# La libreria para "encontrar el sevicio" de Spark
import findspark
findspark.init()

# Librerias para "gestionar el servicio" de Spark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext, SparkSession

# Creamos una aplicación Spark en el Servicio
# Tenga cuidado con las tildes o caracteres especiales en el nombre de la app
AppSpark = SparkConf().setAppName("Evaluacion iSofware")

# definimos un espacio o contexto para la App
ContextoSpark=SparkContext(conf=AppSpark)

# inicio una sesión en el espacio de la App
SesionSpark = SparkSession(ContextoSpark)

# inicio del espacio o contexto SQL
ContextoSql = SQLContext(sparkContext=ContextoSpark, sparkSession=SesionSpark)

In [2]:
!apt-get update && apt-get -y install dos2unix
!pip3 install unidecode
!dos2unix Asistencia\ del\ Pitch\ \(2021.04.13\ ISoftware\).csv
!dos2unix Evaluacion\ del\ Pitch\ \(2021.04.13\ ISoftware\).csv

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic InRelease [242 kB]
Get:3 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2152 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic/universe amd64 Packages [11.3 MB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [24.7 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1412 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [423 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic/restricted amd64 Packages [13.5 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic/main amd64 Packages [1344 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic/multiverse amd64 Packages [186 kB]
Get:13 http://archive.ubuntu

In [3]:
# 0. Modificación de los archivos para tener nombres mucho mas amigables para la manipulación en tablas
# Utilizaremos dos diccionarios para realizar el renombramiento de las columnas

evaluacion_dict = {
    "Marca temporal":                   "timestamp",
    "Nombre de usuario":                "nombre_usuario",
    "Equipo que vas a evaluar:":        "equipo_evaluar",
    "Introducción: El equipo responde adecuadamente ¿Quiénes son y por qué están aquí?":
                                        "introduccion",
    "Equipo: El equipo responde adecuadamente ¿Quiénes están detrás de la idea y cuál es su función?":
                                        "equipo",
    "Problema: El equipo responde adecuadamente ¿Qué problema resolverá?, ¿es realmente un problema?":
                                        "problema",
    "Ventajas: El equipo responde adecuadamente ¿Por qué su solución es especial?, ¿qué la hace distinta de otras?":
                                        "ventajas",
    "Solución: El equipo responde adecuadamente ¿Cómo piensa resolver el problema?":
                                        "solucion",
    "Producto: El equipo responde adecuadamente ¿Cómo funciona el producto o servicio? Muestra algunos ejemplos.":
                                        "producto",
    "Tracción: El equipo responde adecuadamente si cuenta con clientes que demuestran potencial.":
                                        "traccion",
    "Mercado: El equipo responde conoce, o por lo menos intentar predecir, el tamaño del mercado que impactará.":
                                        "mercado",
    "Competencia: El equipo responde adecuadamente ¿Cuáles son las soluciones alternativas al problema que plantea?":
                                        "competencia",
    "Modelo de negocio: El equipo responde adecuadamente ¿Cómo hará dinero? ":
                                        "modelo_negocio",
    "Inversión: El equipo responde adecuadamente ¿Cuál es su presupuesto y cuánto espera ganar?":
                                        "inversion",
    "Contacto: El equipo deja los datos al cliente y muestra cómo pueden contactarle.":
                                        "contacto",
    "Exposición: ¿Qué tan coordinados estaban los expositores?":
                                        "exposicion_cordinacion",
    "Exposición: ¿Los expositores se expresaron con claridad y se hicieron entender?":
                                        "exposicion_entendimiento",
    "Exposición: Las diapositivas son claras y coherentes y apoyaron adecuadamente la exposición.":
                                        "exposicion_diapositivas",
    "Suponiendo que eres inversionista, ¿Estarías dispuesto a invertir dinero en este equipo? (esta pregunta no se pondera en la nota)":
                                        "inversionista",
    "Observaciones para el equipo, estas observaciones las debe considerar el equipo para mejorar la siguiente presentación.":
                                        "observaciones"
}

asistencia_dict = {
    "Marca temporal":                   "timestamp",
    "Nombre de usuario":                "nombre_usuario",
    "Equipo al que perteneces:" :       "equipo_pertenece",
    "Equipo que va a exponer:" :        "equipo_expone"
}

In [4]:
import pandas as pd

asistencia_df = pd.read_csv("Asistencia del Pitch (2021.04.13 ISoftware).csv")
asistencia_df = asistencia_df.rename(columns=asistencia_dict)
asistencia_df.to_csv("Asistencia.csv", index=False, encoding='utf-8')

evaluacion_df = pd.read_csv("Evaluacion del Pitch (2021.04.13 ISoftware).csv")
evaluacion_df = evaluacion_df.rename(columns=evaluacion_dict)
evaluacion_df.to_csv("Evaluacion.csv", index=False, encoding='utf-8')

In [5]:
def remove_accents(filename):
    import unidecode
    with open(filename, 'r+', encoding='utf-8') as f:
        data = f.read()
        data = unidecode.unidecode(data)
        f.seek(0)
        f.write(data)
        f.truncate()

remove_accents('Asistencia.csv')
remove_accents('Evaluacion.csv')

In [6]:
# 1. Cargue los datos en la carpeta datalake y luego del /datalake al HDFS (Hadoop File System)
# Recuerda usar ! para ejecutar el comando en el shell. 
# Tu código a continuación...

!hdfs dfs -mkdir /tmp/input
!hdfs dfs -copyFromLocal -f Asistencia.csv /tmp/input/
!hdfs dfs -copyFromLocal -f Evaluacion.csv /tmp/input/
!hdfs dfs -ls /tmp/input/

Found 2 items
-rw-r--r--   1 root root      86299 2021-05-24 02:11 /tmp/input/Asistencia.csv
-rw-r--r--   1 root root      86157 2021-05-24 02:11 /tmp/input/Evaluacion.csv


In [7]:
# 2. Cree dos tablas SparkSQL y almacene el csv en su correspondiente tabla.
# Observación: tenga especial cuidado con los encabezados de los archivos CSV.
# Usted puede considerar cambiar los encabezados de los CSV originales
# Tu código a continuación...

ContextoSql.sql('DROP TABLE IF EXISTS asistencia');
ContextoSql.sql("""
CREATE TABLE IF NOT EXISTS 
asistencia (
        timestamp TIMESTAMP,
        nombre_usuario STRING,
        equipo_pertenece STRING,
        equipo_expone STRING
        ) 
USING com.databricks.spark.csv 
OPTIONS ( 
    header='true', 
    nullvalue='NA', 
    timestampFormat=\"yyyy/MM/dd hh:mm:ss a\", 
    path '/tmp/input/Asistencia.csv')
""");

ContextoSql.sql('DROP TABLE IF EXISTS evaluacion');
ContextoSql.sql("""
CREATE TABLE IF NOT EXISTS 
evaluacion (
        timestamp TIMESTAMP,
        nombre_usuario STRING,
        equipo_evaluar STRING,
        introduccion INT,
        equipo INT,
        problema INT,
        ventajas INT,
        solucion INT,
        producto INT,
        traccion INT,
        mercado INT,
        competencia INT,
        modelo_negocio INT,
        inversion INT,
        contacto INT,
        exposicion_cordinacion INT,
        exposicion_entendimiento INT,
        exposicion_diapositivas INT,
        inversionista STRING,
        observaciones STRING
        ) 
USING com.databricks.spark.csv 
OPTIONS ( 
    header='true', 
    nullvalue='NA', 
    multiline='true',
    timestampFormat=\"yyyy/MM/dd hh:mm:ss a\", 
    path '/tmp/input/Evaluacion.csv')
""");

ContextoSql.sql('SHOW TABLES').show()

+--------+----------+-----------+
|database| tableName|isTemporary|
+--------+----------+-----------+
| default|asistencia|      false|
| default|evaluacion|      false|
+--------+----------+-----------+



In [8]:
# 2. Consulte el listado total de estudiantes (correos electrónicos) del 
# curso de Ingeniería de Software, ordenados alfabéticamente
# Tu código a continuación...

listado = ContextoSql.sql("SELECT DISTINCT(nombre_usuario) AS estudiante FROM asistencia ORDER BY estudiante")
print(f'Total de estudiantes: {listado.count()}\n\n')
listado.show(100, truncate=False)

Total de estudiantes: 83


+-------------------------+
|estudiante               |
+-------------------------+
|abanolc@unal.edu.co      |
|acastrillonv@unal.edu.co |
|aflemag@unal.edu.co      |
|anapariciom@unal.edu.co  |
|angutierrezb@unal.edu.co |
|anoriega@unal.edu.co     |
|auarbelaeza@unal.edu.co  |
|bocampo@unal.edu.co      |
|cgiraldo@unal.edu.co     |
|cjfunezg@unal.edu.co     |
|cquinchiar@unal.edu.co   |
|dadazam@unal.edu.co      |
|daestradam@unal.edu.co   |
|davgarciava@unal.edu.co  |
|dballesteroso@unal.edu.co|
|dbrito@unal.edu.co       |
|dcadavid@unal.edu.co     |
|dcardonaal@unal.edu.co   |
|dchavarriar@unal.edu.co  |
|dgarciabl@unal.edu.co    |
|dgerenal@unal.edu.co     |
|dgiraldolo@unal.edu.co   |
|dguardia@unal.edu.co     |
|dosoriom@unal.edu.co     |
|eapenad@unal.edu.co      |
|elondonoc@unal.edu.co    |
|emflorezb@unal.edu.co    |
|emlopezr@unal.edu.co     |
|eporrasm@unal.edu.co     |
|esgarciac@unal.edu.co    |
|favasquezj@unal.edu.co   |
|fguerrerot@unal.edu.

In [9]:
# 3. Consulte la cantidad de asistencias registradas por estudiante; además, la fecha y hora de la primera asistencia
# y la fecha y hora de la última asistencia
# Tu código a continuación...

consulta = ContextoSql.sql("""SELECT nombre_usuario, 
                                     COUNT(1) AS asistencias, 
                                     MIN(timestamp) AS primer_asistencia, 
                                     MAX(timestamp) AS ultima_asistencia 
                              FROM asistencia
                              GROUP BY nombre_usuario
                              ORDER BY nombre_usuario""")
consulta.show(truncate=False)

+-------------------------+-----------+-------------------+-------------------+
|nombre_usuario           |asistencias|primer_asistencia  |ultima_asistencia  |
+-------------------------+-----------+-------------------+-------------------+
|abanolc@unal.edu.co      |7          |2021-04-08 08:12:14|2021-04-13 09:37:56|
|acastrillonv@unal.edu.co |8          |2021-04-08 08:13:06|2021-04-13 09:43:13|
|aflemag@unal.edu.co      |8          |2021-04-08 08:11:58|2021-04-13 09:41:54|
|anapariciom@unal.edu.co  |10         |2021-04-08 08:11:49|2021-04-13 09:38:51|
|angutierrezb@unal.edu.co |8          |2021-04-08 08:19:06|2021-04-13 09:43:54|
|anoriega@unal.edu.co     |8          |2021-04-08 08:15:18|2021-04-13 09:40:20|
|auarbelaeza@unal.edu.co  |8          |2021-04-08 08:09:53|2021-04-13 09:41:04|
|bocampo@unal.edu.co      |8          |2021-04-08 08:13:30|2021-04-13 09:42:16|
|cgiraldo@unal.edu.co     |8          |2021-04-08 08:44:23|2021-04-13 09:56:44|
|cjfunezg@unal.edu.co     |8          |2

In [10]:
# 4. Consulte el listado de estudiantes que asistieron a 2 presentaciones o menos (una).
# Tu código a continuación...

consulta = ContextoSql.sql("""SELECT    nombre_usuario, 
                                        COUNT(1) AS asistencias
                              FROM      asistencia
                              GROUP BY  nombre_usuario
                              HAVING    COUNT(1) <= 2
                              ORDER BY nombre_usuario""")
consulta.show(truncate=False)

+---------------------+-----------+
|nombre_usuario       |asistencias|
+---------------------+-----------+
|jdroldano@unal.edu.co|1          |
+---------------------+-----------+



In [11]:
# 5. Consulte el listado de estudiantes que no asistieron a ninguna presentación.
# Tu código a continuación...

consulta = ContextoSql.sql("""SELECT    a.nombre_usuario
                              FROM      asistencia AS a
                              LEFT JOIN evaluacion AS e ON a.nombre_usuario = e.nombre_usuario
                              WHERE     e.nombre_usuario IS NULL
                              ORDER BY  a.nombre_usuario""")
consulta.show(truncate=False)

+--------------+
|nombre_usuario|
+--------------+
+--------------+



In [12]:
# 6. Consulte los integrantes por cada equipo al que pertenecen.
# Tu código a continuación...

import pyspark.sql.functions as f
consulta = ContextoSql.sql("""SELECT    DISTINCT equipo_pertenece,
                                        nombre_usuario
                              FROM      asistencia
                              ORDER BY equipo_pertenece,nombre_usuario""")
consulta = consulta.select(f.regexp_replace('equipo_pertenece', "Pertenezco a: ","").alias('equipo_pertenece'),'nombre_usuario')
consulta.show(100,truncate=False)

+-----------------------------------------------------+-------------------------+
|equipo_pertenece                                     |nombre_usuario           |
+-----------------------------------------------------+-------------------------+
|Administracion de Inmuebles                          |angutierrezb@unal.edu.co |
|Administracion de Inmuebles                          |eapenad@unal.edu.co      |
|Administracion de Inmuebles                          |emflorezb@unal.edu.co    |
|Administracion de Inmuebles                          |framirezf@unal.edu.co    |
|Administracion de Inmuebles                          |jhcordoba@unal.edu.co    |
|Administracion de Inmuebles                          |jortize@unal.edu.co      |
|Administracion de Inmuebles                          |kmolano@unal.edu.co      |
|Administracion de Inmuebles                          |lumarinb@unal.edu.co     |
|Administracion de Inmuebles                          |nsalinas@unal.edu.co     |
|Administracion 

In [13]:
# 7. Consulte la cantidad de asistentes por presentación, sin considerar los asistentes que pertenecen 
# al equipo que realizó la presentación.
# Tu código a continuación...

consulta = ContextoSql.sql("""SELECT DISTINCT 
                                        REPLACE(equipo_expone, 'Expone: ', '') AS equipo_expone,
                                        COUNT(nombre_usuario) AS asistentes
                              FROM      asistencia
                              WHERE     REPLACE(equipo_expone, 'Expone: ', '') !=  REPLACE(equipo_pertenece, 'Pertenezco a: ','')
                              GROUP BY  REPLACE(equipo_expone, 'Expone: ', '')
                              ORDER BY  REPLACE(equipo_expone, 'Expone: ', '')
                              """)
consulta.show(truncate=False)

+-----------------------------------------------------+----------+
|equipo_expone                                        |asistentes|
+-----------------------------------------------------+----------+
|Administracion de Inmuebles                          |70        |
|Desercion de estudiantes                             |3         |
|Gestion de Menus Ejecutivos                          |72        |
|Gestion de Residencias y Habitaciones Universitarias.|72        |
|Gestion de programas curriculares                    |74        |
|Lo tengo y te lo presto.                             |75        |
|Mi Sanandresito                                      |72        |
|Servicios a domicilio                                |76        |
|Te lo cambio.                                        |75        |
+-----------------------------------------------------+----------+



In [14]:
# 8. Consutar cuáles integrantes evaluaron a su propio equipo. Estas evaluaciones no serán válidas, pues un
# integrante no puede evaluar a su propio equipo.
# Tu código a continuación...

consulta = ContextoSql.sql("""SELECT    a.nombre_usuario, 
                                        REPLACE(a.equipo_pertenece, 'Pertenezco a: ','') as equipo_pertenece, 
                                        e.equipo_evaluar
                              FROM      asistencia AS a
                              LEFT JOIN evaluacion AS e ON a.nombre_usuario = e.nombre_usuario
                              WHERE     REPLACE(a.equipo_pertenece, 'Pertenezco a: ','') == e.equipo_evaluar
                              """)
consulta.show(truncate=False)

+-----------------------+---------------------------------+---------------------------------+
|nombre_usuario         |equipo_pertenece                 |equipo_evaluar                   |
+-----------------------+---------------------------------+---------------------------------+
|daestradam@unal.edu.co |Lo tengo y te lo presto.         |Lo tengo y te lo presto.         |
|anapariciom@unal.edu.co|Servicios a domicilio            |Servicios a domicilio            |
|hriveraa@unal.edu.co   |Gestion de Menus Ejecutivos      |Gestion de Menus Ejecutivos      |
|jruedat@unal.edu.co    |Servicios a domicilio            |Servicios a domicilio            |
|lmayar@unal.edu.co     |Gestion de programas curriculares|Gestion de programas curriculares|
|daestradam@unal.edu.co |Lo tengo y te lo presto.         |Lo tengo y te lo presto.         |
|jruedat@unal.edu.co    |Servicios a domicilio            |Servicios a domicilio            |
|hriveraa@unal.edu.co   |Gestion de Menus Ejecutivos      |G

In [15]:
# 9. Consultar la nota promedio por cada ítem (1 al 15), y la nota promedio total del cada equipo. Recuerde que no
# son válidas las evaluaciones realizadas por los miembros del mismo equipo.
# Tu código a continuación...

from pyspark.sql.functions import col, round

resultColumns = [col('introduccion'),
                 col('equipo'), 
                 col('problema'), 
                 col('ventajas'), 
                 col('solucion'), 
                 col('producto'), 
                 col('traccion'), 
                 col('mercado'), 
                 col('competencia'),
                 col('modelo'),
                 col('inversion'),
                 col('contacto'),
                 col('cordinacion'),
                 col('entendimiento'),
                 col('diapositivas'),
                 ]
avgFunc = sum(x for x in resultColumns)/len(resultColumns)

consulta = ContextoSql.sql("""SELECT    e.equipo_evaluar,
                                        ROUND(AVG(introduccion),2) AS introduccion,
                                        ROUND(AVG(equipo),2) AS equipo,
                                        ROUND(AVG(problema),2) AS problema,
                                        ROUND(AVG(ventajas),2) AS ventajas,
                                        ROUND(AVG(solucion),2) AS solucion,
                                        ROUND(AVG(producto),2) AS producto,
                                        ROUND(AVG(traccion),2) AS traccion,
                                        ROUND(AVG(mercado),2) AS mercado,
                                        ROUND(AVG(competencia),2) AS competencia,
                                        ROUND(AVG(modelo_negocio),2) AS modelo,
                                        ROUND(AVG(inversion),2) AS inversion,
                                        ROUND(AVG(contacto),2) AS contacto,
                                        ROUND(AVG(exposicion_cordinacion),2) AS cordinacion,
                                        ROUND(AVG(exposicion_entendimiento),2) AS entendimiento,
                                        ROUND(AVG(exposicion_diapositivas),2) AS diapositivas
                              FROM      asistencia AS a
                              LEFT JOIN evaluacion AS e ON a.nombre_usuario = e.nombre_usuario
                              WHERE     REPLACE(a.equipo_pertenece, 'Pertenezco a: ','') != e.equipo_evaluar
                              GROUP BY  1
                              """)

calificaciones = consulta.withColumn('calificacion_final', round(avgFunc, 2))
calificaciones.show(truncate=False)

+-----------------------------------------------------+------------+------+--------+--------+--------+--------+--------+-------+-----------+------+---------+--------+-----------+-------------+------------+------------------+
|equipo_evaluar                                       |introduccion|equipo|problema|ventajas|solucion|producto|traccion|mercado|competencia|modelo|inversion|contacto|cordinacion|entendimiento|diapositivas|calificacion_final|
+-----------------------------------------------------+------------+------+--------+--------+--------+--------+--------+-------+-----------+------+---------+--------+-----------+-------------+------------+------------------+
|Servicios a domicilio                                |4.49        |4.63  |4.51    |4.27    |4.5     |4.61    |4.1     |4.12   |4.36       |4.27  |4.16     |4.65    |4.49       |4.56         |4.59        |4.42              |
|Lo tengo y te lo presto.                             |4.49        |4.6   |4.27    |4.43    |4.42   

In [16]:
# 10. Consulte el mejor equipo evaluado por cada ítem (según nota promedio. En caso de empate mostrar todos los empatados) 
# y el mejor equipo según el promedio total.
# Tu código a continuación...

columns = calificaciones.columns
columns.pop(0)
columns.pop(-1)

from pyspark.sql.functions import min, max
for column in columns: 
    a = calificaciones.select([max(column)]).collect()[0][0]
    print(f'Equipo mejor evaluado en: {column}')
    calificaciones.select('equipo_evaluar', column).where(calificaciones[column] == a).show(truncate=False)

Equipo mejor evaluado en: introduccion
+-----------------------------------------------------+------------+
|equipo_evaluar                                       |introduccion|
+-----------------------------------------------------+------------+
|Gestion de Residencias y Habitaciones Universitarias.|4.56        |
+-----------------------------------------------------+------------+

Equipo mejor evaluado en: equipo
+------------------------+------+
|equipo_evaluar          |equipo|
+------------------------+------+
|Desercion de estudiantes|5.0   |
+------------------------+------+

Equipo mejor evaluado en: problema
+------------------------+--------+
|equipo_evaluar          |problema|
+------------------------+--------+
|Desercion de estudiantes|5.0     |
+------------------------+--------+

Equipo mejor evaluado en: ventajas
+------------------------+--------+
|equipo_evaluar          |ventajas|
+------------------------+--------+
|Desercion de estudiantes|5.0     |
+---------------

In [17]:
# 11. Consulte el peor equipo evaluado por cada ítem (según nota promedio. En caso de empate mostrar todos los empatados) 
# y el peor equipo según el promedio total.
# Tu código a continuación...

columns = calificaciones.columns
columns.pop(0)
columns.pop(-1)

from pyspark.sql.functions import min, max
for column in columns: 
    a = calificaciones.select([min(column)]).collect()[0][0]
    print(f'Equipo peor evaluado en: {column}')
    calificaciones.select('equipo_evaluar', column).where(calificaciones[column] == a).show(truncate=False)

Equipo peor evaluado en: introduccion
+---------------------------+------------+
|equipo_evaluar             |introduccion|
+---------------------------+------------+
|Administracion de Inmuebles|4.12        |
+---------------------------+------------+

Equipo peor evaluado en: equipo
+---------------+------+
|equipo_evaluar |equipo|
+---------------+------+
|Mi Sanandresito|3.91  |
+---------------+------+

Equipo peor evaluado en: problema
+---------------------------+--------+
|equipo_evaluar             |problema|
+---------------------------+--------+
|Administracion de Inmuebles|3.97    |
+---------------------------+--------+

Equipo peor evaluado en: ventajas
+---------------------------------+--------+
|equipo_evaluar                   |ventajas|
+---------------------------------+--------+
|Gestion de programas curriculares|3.92    |
+---------------------------------+--------+

Equipo peor evaluado en: solucion
+---------------------------+--------+
|equipo_evaluar          

In [18]:
# 12. Consulte la lista de estudiantes con la correspondiente nota obtenida en la presentación 
# (nota promedio total de la evaluación obtenida por el equipo)
# Tu código a continuación...

from pyspark.sql.functions import col, round

resultColumns = [col('introduccion'),
                 col('equipo'), 
                 col('problema'), 
                 col('ventajas'), 
                 col('solucion'), 
                 col('producto'), 
                 col('traccion'), 
                 col('mercado'), 
                 col('competencia'),
                 col('modelo'),
                 col('inversion'),
                 col('contacto'),
                 col('cordinacion'),
                 col('entendimiento'),
                 col('diapositivas'),
                 ]
avgFunc = sum(x for x in resultColumns)/len(resultColumns)

consulta = ContextoSql.sql("""SELECT    e.equipo_evaluar,
                                        ROUND(AVG(introduccion),2) AS introduccion,
                                        ROUND(AVG(equipo),2) AS equipo,
                                        ROUND(AVG(problema),2) AS problema,
                                        ROUND(AVG(ventajas),2) AS ventajas,
                                        ROUND(AVG(solucion),2) AS solucion,
                                        ROUND(AVG(producto),2) AS producto,
                                        ROUND(AVG(traccion),2) AS traccion,
                                        ROUND(AVG(mercado),2) AS mercado,
                                        ROUND(AVG(competencia),2) AS competencia,
                                        ROUND(AVG(modelo_negocio),2) AS modelo,
                                        ROUND(AVG(inversion),2) AS inversion,
                                        ROUND(AVG(contacto),2) AS contacto,
                                        ROUND(AVG(exposicion_cordinacion),2) AS cordinacion,
                                        ROUND(AVG(exposicion_entendimiento),2) AS entendimiento,
                                        ROUND(AVG(exposicion_diapositivas),2) AS diapositivas
                              FROM      asistencia AS a
                              LEFT JOIN evaluacion AS e ON a.nombre_usuario = e.nombre_usuario
                              WHERE     REPLACE(a.equipo_pertenece, 'Pertenezco a: ','') != e.equipo_evaluar
                              GROUP BY  1
                              """)

consulta = consulta.withColumn('calificacion_final', round(avgFunc, 2))
consulta = consulta[['equipo_evaluar','calificacion_final']]

estudiantes = ContextoSql.sql("""SELECT     DISTINCT nombre_usuario AS estudiante,
                                            REPLACE(equipo_pertenece, 'Pertenezco a: ','') AS equipo
                                 FROM       asistencia
                                 """)
#estudiantes.show(100, truncate=False)

calificaciones = estudiantes.join(consulta, estudiantes.equipo == consulta.equipo_evaluar, how='left')
calificaciones[['estudiante', 'equipo', 'calificacion_final']].show(100, truncate=False)


+-------------------------+-----------------------------------------------------+------------------+
|estudiante               |equipo                                               |calificacion_final|
+-------------------------+-----------------------------------------------------+------------------+
|sebermudezg@unal.edu.co  |Servicios a domicilio                                |4.42              |
|jmunozbo@unal.edu.co     |Servicios a domicilio                                |4.42              |
|anapariciom@unal.edu.co  |Servicios a domicilio                                |4.42              |
|bocampo@unal.edu.co      |Servicios a domicilio                                |4.42              |
|sacastrot@unal.edu.co    |Servicios a domicilio                                |4.42              |
|jppinedal@unal.edu.co    |Servicios a domicilio                                |4.42              |
|dosoriom@unal.edu.co     |Servicios a domicilio                                |4.42      